Reload modules every time before executing the Python code typed:

In [1]:
%load_ext autoreload
%autoreload 2

Install necessary packages:

In [2]:
%pip install opencv-python numpy scipy filterpy==1.4.5

Note: you may need to restart the kernel to use updated packages.


# Task 1 (exercise 1.2)

Import necessary packages:

In [3]:
import cv2
import numpy as np
from scipy.stats import linregress

import sys
sys.path.append('./')

from sort import Sort

The higher the value, the bigger a moving object should be to be detected

In [4]:
MIN_CONTOUR_AREA = 4000

In [25]:
# Open the sample video file:
video = cv2.VideoCapture("Traffic_Laramie_1.mp4")

# Ensure that the file was opened correctly:
assert video.isOpened(), "Can't open the video file"

# Initialize an instance of [Gaussian Mixture-based Background/Foreground Segmentation Algorithm](https://docs.opencv.org/3.4/d7/d7b/classcv_1_1BackgroundSubtractorMOG2.html):
bg_subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows = False)

# Instantiate the SORT tracker
tracker = Sort(max_age=5)

cv2.namedWindow("Cam")

counted_car_ids = set()

# Read the video frame by frame:
while True:
    ret, frame = video.read()

    # Check for the last frame
    if not ret:
        break

    # Apply the background subtraction algorithm:
    fg_mask = bg_subtractor.apply(frame)

    # Morphological operations to remove the noise
    kernel = np.ones((4, 4), np.uint8)
    morph = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

    # Find the contours of the detected objects:
    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Prepare bounding boxes for the tracker
    bboxes = []

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)

        # Filter out the small contours
        if cv2.contourArea(contour) < MIN_CONTOUR_AREA:
            continue

        # Main street is in the lower half of the frame
        if y < frame.shape[0] / 2:
            continue

        bboxes.append([x, y, x+w, y+h])
    
    if len(bboxes) > 0:
        detections = tracker.update(np.array(bboxes))
    else:
        # sort.py requires to pass a (0, 5)-shaped empty array if no detections were made
        detections = tracker.update(np.empty((0, 5)))
    
    for t in tracker.trackers:
        # Check if the car was already counted
        # if t.id in counted_car_ids:
        #     continue

        # Check if the car has any position history
        # if not t.history:
        #     continue
        
        # # Check if the car is near the left edge of the frame
        # if t.history[-1][0][0] > 100:
        #     continue

        # Check only the cars with at least 20 tracking updates
        if len(t.history) > 1:
            # The history of a car x-coordinates
            # x_centroids = np.array(t.history)[:,0,2] - np.array(t.history)[:,0,0]
            # cv2.circle(frame, (int(x_centroids[-1]), 10), 5, (255, 0, 0), -1)
            d = t.history[0][-1]
            cv2.rectangle(frame, (int(d[0]), int(d[1])), (int(d[2]), int(d[3])), (255, 0, 0), 5)

            # # Calculate the slope of the linear regression of car's x-centroids to determine the direction of the car
            # slope, _, _, _, _ = linregress(np.arange(len(x_centroids)), x_centroids)

            # # If the slope is negative, the car is going from the right to the left
            # if slope < 0:
            #     counted_car_ids.add(t.id)


    # Draw bounding boxes from the tracking updates
    for d in detections:
        # Display the bounding box around the detected object
        # cv2.rectangle(frame, (int(d[0]), int(d[1])), (int(d[2]), int(d[3])), (0, 255, 0), 2)

        # Display the car tracking ID
        cv2.putText(frame, f"Car #{int(d[4])}", (int(d[0]), int(d[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.putText(frame, f'Number of cars go to the city centre: {len(counted_car_ids)}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow("Cam", frame)

    # Press "q" to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyWindow("Cam")
        cv2.waitKey(1)
        break